In [ ]:

!curl https://ollama.ai/install.sh | sh # Install Ollama AI
!nohup ollama serve & # Start Ollama AI service
!pip install ollama # Install Ollama Python client
!ollama pull llama2 #install model
!pip install streamlit langchain_community


In [10]:
!pip install langchain
!pip install docx2txt
!pip install tiktoken
!ollama pull nomic-embed-text
!pip install chromadb

In [26]:
!pip freeze >> requirements.txt

In [11]:
!pip install pyngrok

In [ ]:
import streamlit as st
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
loader = Docx2txtLoader('About_the_founders.docx')
data = loader.load()
doc_splits = text_splitter.split_documents(data)

from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

from langchain.callbacks.base import BaseCallbackHandler

class StreamingCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.partial_output = ""

    def on_llm_new_token(self, token, **kwargs):
        self.partial_output += token
        print(token, end="", flush=True)

 # URL processing
def process_input(question):
    model_local = Ollama(model="llama2", callbacks=[StreamingCallbackHandler()])

    after_rag_template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
    after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | model_local
        | StrOutputParser()
    )
    return after_rag_chain.invoke(question)

import streamlit as st
from dotenv import load_dotenv
import os
import shelve

load_dotenv()

st.title("Streamlit Chatbot Interface")

USER_AVATAR = "👤"
BOT_AVATAR = "🤖"

# Ensure openai_model is initialized in session state
if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "meta-llama/Llama-2-70b-chat-hf"


# Load chat history from shelve file
def load_chat_history():
    with shelve.open("chat_history") as db:
        return db.get("messages", [])


# Save chat history to shelve file
def save_chat_history(messages):
    with shelve.open("chat_history") as db:
        db["messages"] = messages


# Initialize or load chat history
if "messages" not in st.session_state:
    st.session_state.messages = load_chat_history()

# Sidebar with a button to delete chat history
with st.sidebar:
    if st.button("Delete Chat History"):
        st.session_state.messages = []
        save_chat_history([])

# Display chat messages
for message in st.session_state.messages:
    avatar = USER_AVATAR if message["role"] == "user" else BOT_AVATAR
    with st.chat_message(message["role"], avatar=avatar):
        st.markdown(message["content"])

# Main chat interface
if prompt := st.chat_input("How can I help?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user", avatar=USER_AVATAR):
        st.markdown(prompt)

    with st.chat_message("assistant", avatar=BOT_AVATAR):
        message_placeholder = st.empty()
        full_response = ""
        for response in process_input(prompt):
            full_response += response or ""
            message_placeholder.markdown(full_response + "|")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})

# Save chat history after each interaction
save_chat_history(st.session_state.messages)



In [ ]:
!nohup ollama serve & # Start Ollama AI service
!ollama pull llama2 #install model

In [25]:
from pyngrok import ngrok
import subprocess

ngrok.set_auth_token('2aGUSiqfIzgKx0yh6XHaTX3G11f_5HRDp7atGGPtyxaiH91bc')

port = 8501
streamlit_process = subprocess.Popen(["streamlit", "run", "/content/app.py", f"--server.port={port}"])

# Start ngrok tunnel for the Streamlit app
ngrok_tunnel = ngrok.connect(addr=f'{port}')

# Print the ngrok URL
print("Ngrok Tunnel URL:", ngrok_tunnel.public_url)

# Block until Streamlit process finishes
streamlit_process.wait()


Ngrok Tunnel URL: https://f7c4-35-204-182-23.ngrok-free.app


KeyboardInterrupt: 

In [24]:
process_input('Which school information are you given in context')

The document provided contains information about the department of physics at a university, including the names and qualifications of the faculty members, their experience, and the facilities available in the research center.

In the context of the question "Which school information are you given in the document?", the answer is:

* The school information provided in the document is related to the department of physics at a university.
* The document provides details about the faculty members, including their names, qualifications, and experience.
* It also lists the facilities available in the research center within the department.

Therefore, the school information given in the document is related to the physics department of a university.

'The document provided contains information about the department of physics at a university, including the names and qualifications of the faculty members, their experience, and the facilities available in the research center.\n\nIn the context of the question "Which school information are you given in the document?", the answer is:\n\n* The school information provided in the document is related to the department of physics at a university.\n* The document provides details about the faculty members, including their names, qualifications, and experience.\n* It also lists the facilities available in the research center within the department.\n\nTherefore, the school information given in the document is related to the physics department of a university.'